In [33]:
import os
from pathlib import Path
import requests
import zipfile
import tarfile
import shutil
from warnings import filterwarnings
filterwarnings('ignore')
def download_to_cache(url, filename, force_download=False):
    def _get_custom_cache_dir():
        custom_cache_dir = Path.home() / ".cache"/ "skmini"/ "datasets"
        os.makedirs(custom_cache_dir, exist_ok=True)
        return custom_cache_dir

    file_path = _get_custom_cache_dir() / filename
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        if not force_download:
            print('File Aldready exists. Use force_download=True to force download.')
    else:
        force_download=True

    if force_download:
        try:
            if os.path.isfile(file_path) or os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory and all its contents
                print(f"Successfully deleted the directory and its contents: {file_path}")
        except OSError as e:
            print(f"Error: {e.strerror}")

        response = requests.get(url, stream=True, verify=False)
        print(response)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded file to {file_path}")
        # now that it is downloaded, we might want to decompress the file
        if tarfile.is_tarfile(file_path):
            tar = tarfile.open(file_path)
            os.remove(file_path)
            tar.extractall(path = file_path)
            tar.close()
        elif zipfile.is_zipfile(file_path): # change to endswith, which doesnt require us loading a library
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    print(file_path)
                    os.remove(file_path)
                    os.mkdir(file_path)
                    zip_ref.extractall(file_path)
            except: print('Decompression failed.')


In [34]:
download_to_cache("https://archive.ics.uci.edu/static/public/53/iris.zip", 'iris')

<Response [200]>
Downloaded file to /home/arjun/.cache/skmini/datasets/iris
/home/arjun/.cache/skmini/datasets/iris


some issue between upper and lower download links, but the last one works

In [35]:
download_to_cache("https://github.com/datasciencedojo/datasets/blob/master/titanic.csv", 'titanic.csv')

<Response [200]>
Downloaded file to /home/arjun/.cache/skmini/datasets/titanic.csv


In [36]:
import tarfile
tar = tarfile.open("/home/arjun/.cache/skmini/test.tar", )
tar.extractall(path = "/home/arjun/.cache/skmini/test")
tar.close()


FileNotFoundError: [Errno 2] No such file or directory: '/home/arjun/.cache/skmini/test.tar'